Conduct RFM (recency, frequency, monetary value) analysis on political donors in the UK

In [34]:
#imports
import pandas as pd
import datetime as dt 
import numpy as np

# DATA PROCESSING

In [35]:
#read data
df = pd.read_csv('data/Donations accepted by political parties.csv')
df.shape

/var/folders/9t/ywq9gg3s4zbgn2zw5tfskbv00000gn/T/ipykernel_23597/3107819911.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Donations accepted by political parties.csv')


(65278, 29)

In [36]:

#drop all nans from the columns that we will be using
df = df.dropna(subset=['Value', 'DonorId', 'ReceivedDate'])
df.shape

(62492, 29)

In [37]:
#change received data from string to datetime
df['ReceivedDate'] = pd.to_datetime(df['ReceivedDate'], format='%d/%m/%Y')
df['Value'] = df['Value'].replace('[\£,]', '', regex=True).astype(float)



# RFM ANALYSIS 

In [38]:
#Compute recency values
recency_df = df.groupby(by='DonorId', as_index=False)['ReceivedDate'].max()
recency_df.columns = ['DonorId', 'LastPurchaseDate']
recent_date = recency_df['LastPurchaseDate'].max()
recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(
    lambda x: (recent_date - x).days)
recency_df.head

<bound method NDFrame.head of        DonorId LastPurchaseDate  Recency
0          1.0       2001-05-14     6685
1          2.0       2003-08-11     5866
2          3.0       2015-02-18     1657
3          4.0       2010-12-31     3167
4          5.0       2009-12-15     3548
...        ...              ...      ...
42183  87473.0       2019-08-05       28
42184  87478.0       2018-11-01      305
42185  87479.0       2009-12-29     3534
42186  87480.0       2017-03-13      903
42187  87482.0       2014-02-06     2034

[42188 rows x 3 columns]>

In [39]:
#Compute Frequency 
frequency_df = df.drop_duplicates().groupby(
    by=['DonorId'], as_index=False)['ReceivedDate'].count()
frequency_df.columns = ['DonorId', 'Frequency']
frequency_df.head

<bound method NDFrame.head of        DonorId  Frequency
0          1.0          2
1          2.0          2
2          3.0          3
3          4.0         10
4          5.0          3
...        ...        ...
42183  87473.0          1
42184  87478.0          2
42185  87479.0          1
42186  87480.0          1
42187  87482.0          1

[42188 rows x 2 columns]>

In [40]:
#Compute Monetary Value
monetary_df = df.groupby(by='DonorId', as_index=False)['Value'].sum()
monetary_df.columns = ['DonorId', 'Monetary']
monetary_df.head

<bound method NDFrame.head of        DonorId   Monetary
0          1.0   53000.00
1          2.0  118680.84
2          3.0   26625.00
3          4.0   59495.89
4          5.0    7700.00
...        ...        ...
42183  87473.0    4000.00
42184  87478.0    3529.41
42185  87479.0   11520.00
42186  87480.0  157350.07
42187  87482.0   50000.00

[42188 rows x 2 columns]>

In [49]:
#merge R, F & M values into one df
rfm = pd.merge(recency_df, frequency_df, on='DonorId')
rfm = pd.merge(rfm, monetary_df, on='DonorId')

In [50]:
#rank and normalize RFM values
rfm['R_rank'] = rfm['Recency'].rank(ascending=False)
rfm['F_rank'] = rfm['Frequency'].rank(ascending=True)
rfm['M_rank'] = rfm['Monetary'].rank(ascending=True)
 
# normalizing the rank of the customers
rfm['R_rank_norm'] = (rfm['R_rank']/rfm['R_rank'].max())*100
rfm['F_rank_norm'] = (rfm['F_rank']/rfm['F_rank'].max())*100
rfm['M_rank_norm'] = (rfm['F_rank']/rfm['M_rank'].max())*100
 
rfm.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)

In [51]:
rfm

,DonorId,LastPurchaseDate,Recency,Frequency,Monetary,R_rank_norm,F_rank_norm,M_rank_norm
0,1.0,2001-05-14,6685,2,53000.00,2.093012,92.692235,92.692235
1,2.0,2003-08-11,5866,2,118680.84,12.198967,92.692235,92.692235
2,3.0,2015-02-18,1657,3,26625.00,84.246705,95.422869,95.422869
3,4.0,2010-12-31,3167,10,59495.89,76.621314,98.880013,98.880013
4,5.0,2009-12-15,3548,3,7700.00,65.365744,95.422869,95.422869
...,...,...,...,...,...,...,...,...
42183,87473.0,2019-08-05,28,1,4000.00,99.982222,45.377833,45.377833
42184,87478.0,2018-11-01,305,2,3529.41,97.112923,92.692235,92.692235
42185,87479.0,2009-12-29,3534,1,11520.00,65.753295,45.377833,45.377833
42186,87480.0,2017-03-13,903,1,157350.07,91.110031,45.377833,45.377833


In [52]:
#calculate RFM score
rfm['RFM_Score'] = 0.15*rfm['R_rank_norm']+0.28 * rfm['F_rank_norm']+0.57*rfm['M_rank_norm']
rfm['RFM_Score'] *= 0.05
rfm = rfm.round(2)
rfm[['DonorId', 'RFM_Score']].head


<bound method NDFrame.head of        DonorId  RFM_Score
0          1.0       3.96
1          2.0       4.03
2          3.0       4.69
3          4.0       4.78
4          5.0       4.55
...        ...        ...
42183  87473.0       2.68
42184  87478.0       4.67
42185  87479.0       2.42
42186  87480.0       2.61
42187  87482.0       2.53

[42188 rows x 2 columns]>